In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata
import tqdm
from time import sleep
import time

In [2]:
df = pd.read_hdf("df_prep.h5", key="df")
df

,idn,title,date,lang,urn,title_clean
0,1285990005,"2020 das jahr, das die welt veränderte der ame...",2024,ger,N/A,2020 jahr welt veränderte amerikanische soziol...
1,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...
2,1319076106,aufbau eines key account managements in der to...,2024,ger,urn:nbn:de:101:1-2024021302301350289147,aufbau key account managements tourismusbranch...
4,1315575361,auswirkungen der corona-pandemie auf die ausbi...,2024,ger,urn:nbn:de:101:1-2024011102383134159187,auswirkungen corona-pandemie ausbildung gesund...
5,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...
...,...,...,...,...,...,...
11363,129882589X,"zwei nachbarländer, eine pandemie",2023,ger,urn:nbn:de:101:1-2023080821170040558967,zwei nachbarländer pandemie
11364,1232892661,zwei runden gehen an corona – aber die urologi...,2021,ger,urn:nbn:de:101:1-2021050520504432857162,zwei runden gehen corona urologie ende gewinnen
11365,1307057543,zwischen arbeitserleichterung und de-professio...,2023,ger,urn:nbn:de:101:1-2023102409011131884864,arbeitserleichterung de-professionalisierungsg...
11366,1241532591,zwischen pandemie und wahlkampf: verhältnismäß...,2021,ger,urn:nbn:de:101:1-2021092021390437649681,pandemie wahlkampf verhältnismäßigkeit wahren


# Vorbereitungen:

## Laden spezifischer Bibliotheken: 

In [3]:
import spacy
from spacy import displacy
import os
import plotly.graph_objects as go
import plotly.express as px

In [4]:
#Code für den Download eines der Sprachmodelle: 
#!python -m spacy download de

In [5]:
nlp = spacy.load('de_core_news_sm') ## Laden eines Trained Model for German

In [6]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']


## Tokenization: 

In [7]:
def process_text(text):
    return nlp(text)

In [8]:
df['token'] = df['title_clean'].apply(process_text)
df

,idn,title,date,lang,urn,title_clean,token
0,1285990005,"2020 das jahr, das die welt veränderte der ame...",2024,ger,N/A,2020 jahr welt veränderte amerikanische soziol...,"(2020, jahr, welt, veränderte, amerikanische, ..."
1,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...,"(ängste, allgemeinbevölkerung, zeiten, covid-1..."
2,1319076106,aufbau eines key account managements in der to...,2024,ger,urn:nbn:de:101:1-2024021302301350289147,aufbau key account managements tourismusbranch...,"(aufbau, key, account, managements, tourismusb..."
4,1315575361,auswirkungen der corona-pandemie auf die ausbi...,2024,ger,urn:nbn:de:101:1-2024011102383134159187,auswirkungen corona-pandemie ausbildung gesund...,"(auswirkungen, corona-pandemie, ausbildung, ge..."
5,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...,"(auswirkungen, covid-19-pandemie, primärversor..."
...,...,...,...,...,...,...,...
11363,129882589X,"zwei nachbarländer, eine pandemie",2023,ger,urn:nbn:de:101:1-2023080821170040558967,zwei nachbarländer pandemie,"(zwei, nachbarländer, pandemie)"
11364,1232892661,zwei runden gehen an corona – aber die urologi...,2021,ger,urn:nbn:de:101:1-2021050520504432857162,zwei runden gehen corona urologie ende gewinnen,"(zwei, runden, gehen, corona, urologie, ende, ..."
11365,1307057543,zwischen arbeitserleichterung und de-professio...,2023,ger,urn:nbn:de:101:1-2023102409011131884864,arbeitserleichterung de-professionalisierungsg...,"(arbeitserleichterung, de-professionalisierung..."
11366,1241532591,zwischen pandemie und wahlkampf: verhältnismäß...,2021,ger,urn:nbn:de:101:1-2021092021390437649681,pandemie wahlkampf verhältnismäßigkeit wahren,"(pandemie, wahlkampf, verhältnismäßigkeit, wah..."


## Lemmatization: 

In [9]:
def get_lemma(doc):
    return [(token.lemma_) for token in doc]

In [10]:
df['lemma'] = df['token'].apply(get_lemma)
df

,idn,title,date,lang,urn,title_clean,token,lemma
0,1285990005,"2020 das jahr, das die welt veränderte der ame...",2024,ger,N/A,2020 jahr welt veränderte amerikanische soziol...,"(2020, jahr, welt, veränderte, amerikanische, ...","[2020, Jahr, Welt, verändert, amerikanisch, So..."
1,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...,"(ängste, allgemeinbevölkerung, zeiten, covid-1...","[Angst, Allgemeinbevölkerung, Zeit, covid-19-p..."
2,1319076106,aufbau eines key account managements in der to...,2024,ger,urn:nbn:de:101:1-2024021302301350289147,aufbau key account managements tourismusbranch...,"(aufbau, key, account, managements, tourismusb...","[Aufbau, key, Account, Management, Tourismusbr..."
4,1315575361,auswirkungen der corona-pandemie auf die ausbi...,2024,ger,urn:nbn:de:101:1-2024011102383134159187,auswirkungen corona-pandemie ausbildung gesund...,"(auswirkungen, corona-pandemie, ausbildung, ge...","[auswirkung, corona-pandemie, Ausbildung, gesu..."
5,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...,"(auswirkungen, covid-19-pandemie, primärversor...","[auswirkung, covid-19-pandemie, Primärversorgu..."
...,...,...,...,...,...,...,...,...
11363,129882589X,"zwei nachbarländer, eine pandemie",2023,ger,urn:nbn:de:101:1-2023080821170040558967,zwei nachbarländer pandemie,"(zwei, nachbarländer, pandemie)","[zwei, nachbarländ, Pandemie]"
11364,1232892661,zwei runden gehen an corona – aber die urologi...,2021,ger,urn:nbn:de:101:1-2021050520504432857162,zwei runden gehen corona urologie ende gewinnen,"(zwei, runden, gehen, corona, urologie, ende, ...","[zwei, Runde, gehen, Corona, Urologie, Ende, g..."
11365,1307057543,zwischen arbeitserleichterung und de-professio...,2023,ger,urn:nbn:de:101:1-2023102409011131884864,arbeitserleichterung de-professionalisierungsg...,"(arbeitserleichterung, de-professionalisierung...","[Arbeitserleichterung, de-professionalisierung..."
11366,1241532591,zwischen pandemie und wahlkampf: verhältnismäß...,2021,ger,urn:nbn:de:101:1-2021092021390437649681,pandemie wahlkampf verhältnismäßigkeit wahren,"(pandemie, wahlkampf, verhältnismäßigkeit, wah...","[Pandemie, Wahlkampf, Verhältnismäßigkeit, wahr]"


### Einfache Worthäufigkeitsprüfung: 

In [11]:
# Hier zu suchenden Begriff eingeben: 
term = "corona"  

In [12]:
print(f'"{term}" appears in the text tokens column ' + str(df.token.apply(lambda x: str(x)).apply(lambda x: x.count(term)).sum()) + ' times.')#
print(f'"{term}" appears in the lemmas column ' + str(df['lemma'].apply(lambda x: str(x)).apply(lambda x: x.count(term)).sum()) + ' times.')

"corona" appears in the text tokens column 2597 times.
"corona" appears in the lemmas column 1900 times.


## Optional: Part of speech tags

In [13]:
def get_pos(doc):
    return [(token.pos_, token.tag_) for token in doc]

In [14]:
df['POS'] = df['token'].apply(get_pos)
df.head()

,idn,title,date,lang,urn,title_clean,token,lemma,POS
0,1285990005,"2020 das jahr, das die welt veränderte der ame...",2024,ger,N/A,2020 jahr welt veränderte amerikanische soziol...,"(2020, jahr, welt, veränderte, amerikanische, ...","[2020, Jahr, Welt, verändert, amerikanisch, So...","[(NUM, CARD), (NOUN, NN), (NOUN, NN), (ADJ, AD..."
1,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...,"(ängste, allgemeinbevölkerung, zeiten, covid-1...","[Angst, Allgemeinbevölkerung, Zeit, covid-19-p...","[(ADJ, ADJA), (NOUN, NN), (NOUN, NN), (PROPN, ..."
2,1319076106,aufbau eines key account managements in der to...,2024,ger,urn:nbn:de:101:1-2024021302301350289147,aufbau key account managements tourismusbranch...,"(aufbau, key, account, managements, tourismusb...","[Aufbau, key, Account, Management, Tourismusbr...","[(NOUN, NN), (ADP, APPR), (PROPN, NE), (PROPN,..."
4,1315575361,auswirkungen der corona-pandemie auf die ausbi...,2024,ger,urn:nbn:de:101:1-2024011102383134159187,auswirkungen corona-pandemie ausbildung gesund...,"(auswirkungen, corona-pandemie, ausbildung, ge...","[auswirkung, corona-pandemie, Ausbildung, gesu...","[(ADJ, ADJA), (NOUN, NN), (NOUN, NN), (VERB, V..."
5,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...,"(auswirkungen, covid-19-pandemie, primärversor...","[auswirkung, covid-19-pandemie, Primärversorgu...","[(ADJ, ADJA), (ADJ, FM), (NOUN, NN), (ADJ, ADJ..."


# Topic Modeling:

### Latent Dirichlet Allocation (LDA):¶

In [15]:
import gensim
from gensim import corpora

In [16]:
lemmatized_text = df.token.apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))


In [17]:
# Create a dictionary and document-term matrix for topic modeling
dictionary = corpora.Dictionary(lemmatized_text.apply(lambda x: x.split()))
doc_term_matrix = [dictionary.doc2bow(doc.split()) for doc in lemmatized_text]

In [18]:
# Apply LDA for topic modeling
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=3, id2word=dictionary, passes=15)

In [19]:
# Visualize the topics
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.184111  0.016684       1        1  50.562117
2      0.075729 -0.149451       2        1  24.932096
1      0.108382  0.132767       3        1  24.505787, topic_info=                  Term         Freq        Total Category  logprob  loglift
16                  --  4525.000000  4525.000000  Default  30.0000  30.0000
4             Pandemie  1274.000000  1274.000000  Default  29.0000  29.0000
11                Zeit   510.000000   510.000000  Default  28.0000  28.0000
19              Corona   354.000000   354.000000  Default  27.0000  27.0000
13   covid-19-pandemie   557.000000   557.000000  Default  26.0000  26.0000
..                 ...          ...          ...      ...      ...      ...
129             sozial    41.188994   173.658693   Topic3  -5.8981  -0.0327
144         sars-cov-2    34.069039    93.921376   Topic3  -6.0879   0.3922
557               2021    35.526959   164.466906   Topic3  -6.0460  -0.1262
775            Zukunft    32.516187    82.599597   Topic3  -6.1346   0.4740
463       Wissenschaft    31.806695    98.950256   Topic3  -6.1566   0.2713

[193 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
16        1  0.933997          --
16        2  0.028283          --
16        3  0.037784          --
0         1  0.503193        2020
0         2  0.369008        2020
...     ...       ...         ...
91        2  0.346354       die
91        3  0.138001       die
694       3  0.990731           §
293       3  0.976174  Österreich
901       3  0.957148  ökologisch

[217 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])